# 6.2.2 Despliegue de modelos

Cuando hablamos de despliegue de modelos o puesta en producción nos estamos refieriendo no solo a la creación y validación de un modelo, sino de mantenerlo activo en un servidor y reentrenarlo cuando tengamos datos nuevos para que el cliente tenga la información lo más actualizada posible y pueda aplicarlo de la forma más segura posible, por lo tanto estamos aplicando una nueva etapa a nuestro trabajo como Científicos o Analistas de Datos. 

![MLOPS](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/11/results___11_0.png)

Como siempre lo primero que debemos hacer es importar las libreías necesarias para poder realizar nuestro trabajo

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer as cancer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score



## Carga de Datos y selección de características

En este caso tenemos datos sobre la detección de Cancer de Mama

In [ ]:
data = cancer()

In [ ]:
print(data.DESCR)

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)

In [ ]:
y = pd.DataFrame(data.target, columns=['Target'])

In [ ]:
df.shape, y.shape

In [ ]:
df.describe().T

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(data = df, x='mean radius', y='mean symmetry', hue=y.Target);
plt.title(f' 0={data.target_names[0]}, 1={data.target_names[1]}');

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(data = y, x='Target')
plt.title(f' 0={data.target_names[0]}, 1={data.target_names[1]}');

Como podemos observar únicamente con dos variables podríamos hacer un clasificador con un un acierto previsiblemente alto, la elección de estas caracteristicas no ha sido aleatoria, se debe al estudio y al conocimiento del problema, es conocido que cuanto mayor sea el radio de un tumor y cuanto más asimétrico sea, mayor es la probabilidad de que este sea de caracter maligno.
Pero con la intención de mejorar la precisión del modelo vamos ha seleccionar algunas variables más como son la textura y su compacticidad, ya que son indicadores de si el tumor puede estar encapsulado o no, otro indicador que nos ayudará a identificar la posibilidad de que sea maligno o no.

## Train test split

In [ ]:
X = df[['mean radius','mean symmetry', 'mean compactness','mean texture']]
y = y.Target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.2, stratify=y)

## Balanceo de clases

In [ ]:
smote = SMOTE()

In [ ]:
X_train, y_train = smote.fit_resample(X_train, y_train)

## Escalado de los datos (normalización)

In [ ]:
mm = MinMaxScaler().fit(X_train)

Debemos recordar aplicar el entreno de nuestro escaler y de SMOTE solo sobre el set de entrenamiento, para mantener a nuestro modelo lejos del set de validación y esté libre de posibles sesgos.

In [ ]:
X_train = mm.transform(X_train)
X_test = mm. transform(X_test)

## Entrenamiento de modelos

In [ ]:
lr = LogisticRegression(max_iter=2000)
rf = RandomForestClassifier()

models = [lr, rf]

In [ ]:
res = {}
for model in models:
    name = str(model).split('(')[0]
    
    model.fit(X_train, y_train)
    
    preds = model.predict(X_train)
    preds_test = model.predict(X_test)
    
    acc = accuracy_score(y_train, preds)
    rec = recall_score(y_train, preds)
    prec = precision_score(y_train, preds)
    f1 = f1_score(y_train, preds)
    
    acc_t = accuracy_score(y_test, preds_test)
    rec_t = recall_score(y_test, preds_test)
    prec_t = precision_score(y_test, preds_test)
    f1_t = f1_score(y_test, preds_test)
    
    res[name]={'Accuracy Train': acc,
               'Recall Tain': rec,
               'Precision Train': prec,
               'f1 Train': f1,
               'Accuracy Test': acc_t,
               'Recall Test': rec_t,
               'Precision Test': prec_t,
               'f1 test': f1_t}
    
    print(res[name])
    
    plt.figure(figsize=(12,8))
    sns.heatmap(confusion_matrix(y_test, preds_test), annot=True)
    plt.title(f'Confusion Matrix {name} Test')

In [ ]:
pd.DataFrame(res)

Una vez tenemos nuestro ajustado podemos importarlo para poder exportarlo a nuestro entorno de producción, para ello usaremos la librería pickle que no permite exportar objetos de python y poder importarlos en diferentes entornos de producción.

In [ ]:
pickle.dump(lr, open('src/logisticregression.pkl', 'wb'))
pickle.dump(mm, open('src/scaler.pkl', 'wb'))